In [5]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import re
import scanpy as sc
import umap
import anndata
# import tables
import scipy.sparse as sp
import seaborn as sns
from matplotlib import cm, colors
from matplotlib.pyplot import rc_context
import sklearn

In [2]:
!ls ../data

e18_mouse_brain_fresh_5k_filtered_feature_bc_matrix.h5


In [38]:
adata1 = sc.read_h5ad(
    '/Users/ninaxiong/Downloads/ENCFF138QCP.h5ad'
)

# adata1 = adata1.var_names_make_unique()
genes = [gene.upper() for gene in list(adata1.var["gene_symbols"])]
len(genes)

69690

In [14]:
gencode = pd.read_table("/Users/ninaxiong/Downloads/gencode.v42.annotation.gff3", comment="#",
                        sep = "\t", names = ['seqname', 'source', 'feature', 'start' , 'end', 'score', 'strand', 'frame', 'attribute'])
gencode.head()

,seqname,source,feature,start,end,score,strand,frame,attribute
0,chr1,HAVANA,gene,11869,14409,.,+,.,ID=ENSG00000290825.1;gene_id=ENSG00000290825.1...
1,chr1,HAVANA,transcript,11869,14409,.,+,.,ID=ENST00000456328.2;Parent=ENSG00000290825.1;...
2,chr1,HAVANA,exon,11869,12227,.,+,.,ID=exon:ENST00000456328.2:1;Parent=ENST0000045...
3,chr1,HAVANA,exon,12613,12721,.,+,.,ID=exon:ENST00000456328.2:2;Parent=ENST0000045...
4,chr1,HAVANA,exon,13221,14409,.,+,.,ID=exon:ENST00000456328.2:3;Parent=ENST0000045...


In [15]:
gencode.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3411685 entries, 0 to 3411684
Data columns (total 9 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   seqname    object
 1   source     object
 2   feature    object
 3   start      int64 
 4   end        int64 
 5   score      object
 6   strand     object
 7   frame      object
 8   attribute  object
dtypes: int64(2), object(7)
memory usage: 234.3+ MB


In [27]:
gencode_genes = gencode[(gencode.feature == "gene")][['seqname', 'start', 'end', 'attribute']].copy().reset_index().drop('index', axis=1)
gencode_genes.head()

,seqname,start,end,attribute
0,chr1,11869,14409,ID=ENSG00000290825.1;gene_id=ENSG00000290825.1...
1,chr1,12010,13670,ID=ENSG00000223972.6;gene_id=ENSG00000223972.6...
2,chr1,14404,29570,ID=ENSG00000227232.5;gene_id=ENSG00000227232.5...
3,chr1,17369,17436,ID=ENSG00000278267.1;gene_id=ENSG00000278267.1...
4,chr1,29554,31109,ID=ENSG00000243485.5;gene_id=ENSG00000243485.5...


In [28]:
def gene_info(x):
    # Extract gene names
    g_name = list(filter(lambda x: 'gene_name' in x,  x.split(";")))[0].split("=")[1]
    g_type = list(filter(lambda x: 'gene_type' in x,  x.split(";")))[0].split("=")[1]
    g_leve = int(list(filter(lambda x: 'level' in x,  x.split(";")))[0].split("=")[1])
    return (g_name, g_type, g_leve)

In [29]:
gencode_genes["gene_name"], gencode_genes["gene_type"], gencode_genes["gene_level"] = zip(
    *gencode_genes.attribute.apply(lambda x: gene_info(x)))
gencode_genes.head()

,seqname,start,end,attribute,gene_name,gene_type,gene_level
0,chr1,11869,14409,ID=ENSG00000290825.1;gene_id=ENSG00000290825.1...,DDX11L2,lncRNA,2
1,chr1,12010,13670,ID=ENSG00000223972.6;gene_id=ENSG00000223972.6...,DDX11L1,transcribed_unprocessed_pseudogene,2
2,chr1,14404,29570,ID=ENSG00000227232.5;gene_id=ENSG00000227232.5...,WASH7P,unprocessed_pseudogene,2
3,chr1,17369,17436,ID=ENSG00000278267.1;gene_id=ENSG00000278267.1...,MIR6859-1,miRNA,3
4,chr1,29554,31109,ID=ENSG00000243485.5;gene_id=ENSG00000243485.5...,MIR1302-2HG,lncRNA,2


In [43]:
filtered = gencode_genes[gencode_genes["gene_name"].str.upper().isin(genes)]
filtered = filtered[["seqname", "start", "end", "gene_name"]]
len(filtered)

15889

In [48]:
filtered.to_csv("../data/gene_locations.tsv", sep="\t", index=False)